# Assignment 2: use SageMaker processing and training jobs
In this assignment you move your data processing, feature enginering, and model training code to SageMaker jobs.

The following diagram shows an anatomy of a SageMaker container:

![](../img/container-anatomy.png)

Refer to the notebook [`02-sagemaker-containers.ipynb`](../02-sagemaker-containers.ipynb) for code snippets and a general guidance for the exercises in this assignment.

## Import packages

In [1]:
import time
import boto3
import botocore
import numpy as np  
import pandas as pd  
import sagemaker
import os
import mlflow
from time import gmtime, strftime, sleep
from sagemaker.processing import FrameworkProcessor, ProcessingInput, ProcessingOutput
from sagemaker.sklearn.estimator import SKLearn
from sklearn.metrics import roc_auc_score
from mlflow import MlflowClient
from IPython.display import Javascript
from importlib.metadata import version, PackageNotFoundError
from IPython.display import HTML

(sagemaker.__version__, boto3.__version__, mlflow.__version__)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


('2.253.1', '1.40.51', '2.22.2')

In [2]:
session = sagemaker.Session()
sm = session.sagemaker_client

## [Optional] Load an existing or create a new experiment
Load an existing or create a new experiment to track parameters, metrics, and artifacts in this notebook.

In [5]:
# # Load experiment based on the a name
# # experiment = Experiment.load(experiment_name, sagemaker_boto_client=sm)
# experiment_suffix = strftime('%d-%H-%M-%S', gmtime())
# registered_model_name = f"from-idea-to-prod-job-model-{experiment_suffix}"

# experiment_name = f"from-idea-to-prod-experiment-{experiment_suffix}"

# print(f'Using MLflow server: {'http://localhost:5000'}')
# mlflow.set_tracking_uri('http://localhost:5000')
# experiment = mlflow.set_experiment(experiment_name=experiment_name)

Using MLflow server: http://localhost:5000


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/urllib3/connection.py:174 in _new_conn                   │
│                                                                                                  │
│   171 │   │   │   extra_kw["socket_options"] = self.socket_options                               │
│   172 │   │                                                                                      │
│   173 │   │   try:                                                                               │
│ ❱ 174 │   │   │   conn = connection.create_connection(                                           │
│   175 │   │   │   │   (self._dns_host, self.port), self.timeout, **extra_kw                      │
│   176 │   │   │   )                                                                              │
│   177                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/urllib3/util/connection.py:95 in create_connection       │
│                                                                                                  │
│    92 │   │   │   │   sock = None                                                                │
│    93 │                                                                                          │
│    94 │   if err is not None:                                                                    │
│ ❱  95 │   │   raise err                                                                          │
│    96 │                                                                                          │
│    97 │   raise socket.error("getaddrinfo returns an empty list")                                │
│    98                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/urllib3/util/connection.py:85 in create_connection       │
│                                                                                                  │
│    82 │   │   │   │   sock.settimeout(timeout)                                                   │
│    83 │   │   │   if source_address:                                                             │
│    84 │   │   │   │   sock.bind(source_address)                                                  │
│ ❱  85 │   │   │   sock.connect(sa)                                                               │
│    86 │   │   │   return sock                                                                    │
│    87 │   │                                                                                      │
│    88 │   │   except socket.error as e:                                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /opt/conda/lib/python3.12/site-packages/urllib3/connectionpool.py:715 in urlopen                 │
│                                                                                                  │
│    712 │   │   │   │   self._prepare_proxy(conn)                                                 │
│    713 │   │   │                                                                                 │
│    714 │   │   │   # Make the request on the httplib connection object.                          │
│ ❱  715 │   │   │   httplib_response = self._make_request(                                        │
│    716 │   │   │   │   conn,       

In [ ]:
# Alternatively, create a new experiment
experiment_name = f"from-idea-to-prod-experiment-{strftime('%d-%H-%M-%S', gmtime())}"
experiment = Experiment.create(
   experiment_name=experiment_name,
   description="Direct marketing binary classification",
   sagemaker_boto_client=sm,
)

## Excercise 1: Process data
- Use SageMaker session object to [upload](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.upload_data)  the dataset to an Amazon S3 bucket. Use a SageMaker [default bucket](https://sagemaker.readthedocs.io/en/stable/api/utility/session.html#sagemaker.session.Session.default_bucket)
- Move data processing code from the previous notebook to a Python executable script. You can pass any parameters to your script to parametrize the data processing
- Set the Amazon S3 paths for the output datasets
- Use [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/overview.html) [`SKLearnProcessor`](https://sagemaker.readthedocs.io/en/stable/frameworks/sklearn/sagemaker.sklearn.html#sagemaker.sklearn.processing.SKLearnProcessor) class to setup a processing job. 
- Configure processing job's [inputs](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingInput) and [outputs](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ProcessingOutput) to point the processing job to Amazon S3 locations
- [Run](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run) the processing job

### Python SDK processor classes
Use the most suitable class to implement a processor for your use case:
    
![](../img/python-sdk-processors.png)

In [ ]:
session = sagemaker.Session()

In [ ]:
# Write data upload code
# S3 key to the full dataset
# input_s3_url = session.upload_data()

input_s3_url = session.upload_data(
    path="data/bank-additional/bank-additional-full.csv",
    bucket=bucket_name,
    key_prefix=f"{bucket_prefix}/input"
)

%store input_s3_url

In [ ]:
%%writefile processing/preprocessing.py

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
import pandas as pd
import numpy as np
import argparse
import os

from time import gmtime, strftime, sleep
import traceback

import mlflow

user_profile_name = os.getenv('USER')

def _parse_args():
    
    parser = argparse.ArgumentParser()
    # Data, model, and output directories
    # model_dir is always passed in from SageMaker. By default this is a S3 path under the default bucket.
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='bank-additional-full.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    
    return parser.parse_known_args()

def process_data(df_data):
    # Indicator variable to capture when pdays takes a value of 999
    df_data["no_previous_contact"] = np.where(df_data["pdays"] == 999, 1, 0)

    # Indicator for individuals not actively employed
    df_data["not_working"] = np.where(
        np.in1d(df_data["job"], ["student", "retired", "unemployed"]), 1, 0
    )

    # remove unnecessary data
    df_model_data = df_data.drop(
        ["duration", "emp.var.rate", "cons.price.idx", "cons.conf.idx", "euribor3m", "nr.employed"],
        axis=1,
    )

    bins = [18, 30, 40, 50, 60, 70, 90]
    labels = ['18-29', '30-39', '40-49', '50-59', '60-69', '70-plus']

    df_model_data['age_range'] = pd.cut(df_model_data.age, bins, labels=labels, include_lowest=True)
    df_model_data = pd.concat([df_model_data, pd.get_dummies(df_model_data['age_range'], prefix='age', dtype=int)], axis=1)
    df_model_data.drop('age', axis=1, inplace=True)
    df_model_data.drop('age_range', axis=1, inplace=True)

    scaled_features = ['pdays', 'previous', 'campaign']
    df_model_data[scaled_features] = MinMaxScaler().fit_transform(df_model_data[scaled_features])

    df_model_data = pd.get_dummies(df_model_data, dtype=int)  # Convert categorical variables to sets of indicators

    # Replace "y_no" and "y_yes" with a single label column, and bring it to the front:
    df_model_data = pd.concat(
        [
            df_model_data["y_yes"].rename(target_col),
            df_model_data.drop(["y_no", "y_yes"], axis=1),
        ],
        axis=1,
    )
    
    return df_model_data

if __name__=="__main__":
    # Process arguments
    args, _ = _parse_args()
    
    target_col = "y"

    # Set the Tracking Server URI using the ARN of the Tracking Server you created
    mlflow.set_tracking_uri(os.environ['MLFLOW_TRACKING_ARN'])
    
    # Enable autologging in MLflow
    mlflow.autolog()

    # Use the active run_id to log 
    with mlflow.start_run(run_id=os.environ['MLFLOW_RUN_ID']) as run:
        # process data
        df_model_data = process_data(pd.read_csv(os.path.join(args.filepath, args.filename), sep=";"))
    
        # Shuffle and splitting dataset
        train_data, validation_data, test_data = np.split(
            df_model_data.sample(frac=1, random_state=1729),
            [int(0.7 * len(df_model_data)), int(0.9 * len(df_model_data))],
        )
    
        print(f"Data split > train:{train_data.shape} | validation:{validation_data.shape} | test:{test_data.shape}")
        mlflow.log_params(
            {
                "train": train_data.shape,
                "validate": validation_data.shape,
                "test": test_data.shape
            }
        )

        mlflow.set_tags(
            {
                'mlflow.user':user_profile_name,
                'mlflow.source.type':'JOB'
            }
        )
        
        # Save datasets locally
        train_data.to_csv(os.path.join(args.outputpath, 'train/train.csv'), index=False, header=False)
        validation_data.to_csv(os.path.join(args.outputpath, 'validation/validation.csv'), index=False, header=False)
        test_data[target_col].to_csv(os.path.join(args.outputpath, 'test/test_y.csv'), index=False, header=False)
        test_data.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'test/test_x.csv'), index=False, header=False)
        
        # Save the baseline dataset for model monitoring
        df_model_data.drop([target_col], axis=1).to_csv(os.path.join(args.outputpath, 'baseline/baseline.csv'), index=False, header=False)

        mlflow.log_artifact(local_path=os.path.join(args.outputpath, 'baseline/baseline.csv'))
    
    print("## Processing complete. Exiting.")

In [ ]:
import sagemaker

session = sagemaker.Session()
bucket_name = session.default_bucket()  # Automatically gets the default SageMaker bucket
bucket_prefix = session.default_bucket_prefix


In [ ]:
# Set the Amazon S3 paths for the output datasets
train_s3_url = f"s3://{bucket_name}/{bucket_prefix}/train"
validation_s3_url = f"s3://{bucket_name}/{bucket_prefix}/validation"
test_s3_url = f"s3://{bucket_name}/{bucket_prefix}/test"
baseline_s3_url = f"s3://{bucket_name}/{bucket_prefix}/baseline"


%store train_s3_url
%store validation_s3_url
%store test_s3_url
%store baseline_s3_url

### [Optional] Create a trail
If you use an experiment, you must create a trial to capture processing and training output from this notebook. 

Use [`Trial`](https://sagemaker-experiments.readthedocs.io/en/latest/trial.html) class to interact with trials and the [`Tracker`](https://sagemaker-experiments.readthedocs.io/en/latest/tracker.html) class to record information to a trial component. 

SageMaker processing and training jobs automatically handle trial components and save metrics, parameters, metadata, and artifacts in the trial components if you provide an `experiment_config` in `Processor.run()` or `Estimator.fit()` calls.

In [ ]:
trial = experiment.create_trial(trial_name_prefix="Container-training")

In [ ]:
with Tracker.create(display_name="Preprocessing-split", sagemaker_boto_client=sm) as tracker:
   tracker.log_parameters()
   tracker.log_input()

In [ ]:
Create experiment config to use in the processing and training jobs
experiment_config = {
   "ExperimentName": experiment.experiment_name,
   "TrialName": trial.trial_name,
   "TrialComponentDisplayName": "Preprocessing",
}

### Create a processor

In [8]:
# Create SKLearnProcessor
# framework_version = "0.23-1"
# processing_instance_type = "ml.m5.large"
# processing_instance_count = 1

sklearn_processor = SKLearnProcessor(
    framework_version = "0.23-1",
    instance_type = "ml.m5.large",
    role=sagemaker.get_execution_role(),
    instance_count = 1,
)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:6                                                                                    │
│                                                                                                  │
│    3 # processing_instance_type = "ml.m5.large"                                                  │
│    4 # processing_instance_count = 1                                                             │
│    5                                                                                             │
│ ❱  6 sklearn_processor = SKLearnProcessor(                                                       │
│    7 │   framework_version = "0.23-1",                                                           │
│    8 │   instance_type = "ml.m5.large",                                                          │
│    9 │   role=sagemaker.get_execution_role(),                                                    │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'SKLearnProcessor' is not defined

In [6]:
from sagemaker.processing import ProcessingInput, ProcessingOutput

processing_inputs = [
    ProcessingInput(
        source="s3://sagemaker-us-east-1-284038851123/from-idea-to-prod/xgboost/input/bank-additional-full.csv",
        destination="/opt/ml/processing/input"
    )
]

processing_outputs = [
    ProcessingOutput(
        source="/opt/ml/processing/output",  # or your actual output directory
        destination="s3://sagemaker-studio-284038851123-prj0b7d45ps/Output/"
    )
]

In [7]:
# Start the processing job, pass an experiment_config parameter if you use experiments
sklearn_processor.run(
    inputs=processing_inputs,
    outputs=processing_outputs,
    code="processing/preprocessing.py"
)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 # Start the processing job, pass an experiment_config parameter if you use experiments       │
│ ❱ 2 sklearn_processor.run(                                                                       │
│   3 │   inputs=processing_inputs,                                                                │
│   4 │   outputs=processing_outputs,                                                              │
│   5 │   code="processing/preprocessing.py"                                                       │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'sklearn_processor' is not defined

In [ ]:
mlflow.set_tags(
    {
        'mlflow.source.name':f'https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/processing-jobs/{sklearn_processor.latest_job.name}',
    }
)

mlflow.end_run()

## Exercise 2: Model training
- Get a container image URI for the used built-in SageMaker ML algorithm using SageMaker SDK [helper](https://sagemaker.readthedocs.io/en/stable/api/utility/image_uris.html#sagemaker.image_uris.retrieve)
- Configure data [input channels](https://sagemaker.readthedocs.io/en/stable/api/utility/inputs.html#sagemaker.inputs.TrainingInput) for the training job
- Use [`Estimator`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator) class to setup a training job
- Set [hyperparameters](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Estimator.set_hyperparameters)
- [Run](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.fit) the training job

In [43]:
# Write code to retrieve a container image URI
!mkdir -p ./training/
!sudo rm -rf ./training-local/
!mkdir -p ./training-local/
%pip show xgboost

Name: xgboost
Version: 2.1.4
Summary: XGBoost Python Package
Home-page: 
Author: 
Author-email: Hyunsu Cho <chohyu01@cs.washington.edu>, Jiaming Yuan <jm.yuan@outlook.com>
License: Apache-2.0
Location: /opt/conda/lib/python3.12/site-packages
Requires: numpy, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [56]:
%%writefile ./training/train.py

import argparse
import json
import logging
import os
import pandas as pd
import pickle as pkl

from sagemaker_containers import entry_point
from sagemaker_xgboost_container.data_utils import get_dmatrix
from sagemaker_xgboost_container import distributed

from sklearn.metrics import roc_auc_score

import xgboost as xgb
import mlflow

from time import gmtime, strftime

suffix = strftime('%d-%H-%M-%S', gmtime())

user_profile_name = os.getenv('USER', 'sagemaker')
experiment_name = os.getenv('MLFLOW_EXPERIMENT_NAME')
region = os.getenv('REGION')

mlflow.set_tracking_uri(os.getenv('MLFLOW_TRACKING_ARN'))
mlflow.set_experiment(experiment_name=experiment_name if experiment_name else f"train-{suffix}")

def _xgb_train(params, dtrain, dval, evals, num_boost_round, model_dir, is_master):
    """Run xgb train on arguments given with rabit initialized.

    This is our rabit execution function.

    :param args_dict: Argument dictionary used to run xgb.train().
    :param is_master: True if current node is master host in distributed training,
                        or is running single node training job.
                        Note that rabit_run includes this argument.
    """
    booster = xgb.train(
        params=params,
        dtrain=dtrain,
        evals=evals,
        num_boost_round=num_boost_round
    )

    val_auc = roc_auc_score(dval.get_label(), booster.predict(dval))
    train_auc = roc_auc_score(dtrain.get_label(), booster.predict(dtrain))
    mlflow.log_params(params)
    mlflow.log_metrics({"validation_auc":val_auc, "train_auc":train_auc})
    # emit training metrics - SageMaker collects them from the log stream
    print(f"[0]#011train-auc:{train_auc}#011validation-auc:{val_auc}")
    
    if is_master:
        model_location = model_dir + '/xgboost-model'
        pkl.dump(booster, open(model_location, 'wb'))
        print("Stored trained model at {}".format(model_location))


if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    
    # Hyperparameters are described here.
    parser.add_argument('--max_depth', type=int)
    parser.add_argument('--eta', type=float)
    parser.add_argument('--alpha', type=float)
    parser.add_argument('--gamma', type=int)
    parser.add_argument('--min_child_weight', type=float)
    parser.add_argument('--subsample', type=float)
    parser.add_argument('--colsample_bytree', type=float)
    parser.add_argument('--verbosity', type=int)
    parser.add_argument('--objective', type=str)
    parser.add_argument('--num_round', type=int)
    parser.add_argument('--early_stopping_rounds', type=int)
    parser.add_argument('--tree_method', type=str, default="auto")
    parser.add_argument('--predictor', type=str, default="auto")

    # Sagemaker specific arguments. Defaults are set in the environment variables.
    parser.add_argument('--output_data_dir', type=str, default=os.environ.get('SM_OUTPUT_DATA_DIR'))
    parser.add_argument('--model_dir', type=str, default=os.environ.get('SM_MODEL_DIR'))
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAIN'))
    parser.add_argument('--validation', type=str, default=os.environ.get('SM_CHANNEL_VALIDATION'))
    parser.add_argument('--sm_hosts', type=str, default=os.environ.get('SM_HOSTS'))
    parser.add_argument('--sm_current_host', type=str, default=os.environ.get('SM_CURRENT_HOST'))
    parser.add_argument('--sm_training_env', type=str, default=os.environ.get('SM_TRAINING_ENV'))
    
    print("main function")
    args, _ = parser.parse_known_args()

    # Get SageMaker host information from runtime environment variables
    sm_hosts = json.loads(args.sm_hosts)
    sm_current_host = args.sm_current_host
    dtrain = get_dmatrix(args.train, 'CSV')
    dval = get_dmatrix(args.validation, 'CSV')

    watchlist = [(dtrain, 'train'), (dval, 'validation')] if dval is not None else [(dtrain, 'train')]

    # get SageMaker enviroment setup
    sm_training_env = json.loads(args.sm_training_env)
    
    # enable auto logging
    mlflow.xgboost.autolog(log_model_signatures=False, log_datasets=False)

    train_hp = {
        'max_depth': args.max_depth,
        'eta': args.eta,
        'gamma': args.gamma,
        'min_child_weight': args.min_child_weight,
        'subsample': args.subsample,
        'verbosity': args.verbosity,
        'objective': args.objective,
        'tree_method': args.tree_method,
        'predictor': args.predictor,
    }

    xgb_train_args = dict(
        params=train_hp,
        dtrain=dtrain,
        dval=dval,
        evals=watchlist,
        num_boost_round=args.num_round,
        model_dir=args.model_dir)

    with mlflow.start_run(
        run_name=f"container-training-{suffix}",
        description="xgboost running in SageMaker container in script mode"
    ) as run:

        mlflow.set_tags(
            {
                'mlflow.user':user_profile_name,
                'mlflow.source.type':'JOB',
                'mlflow.source.name': f"https://{region}.console.aws.amazon.com/sagemaker/home?region={region}#/jobs/{sm_training_env['job_name']}" if sm_training_env['current_host'] != 'sagemaker-local' else sm_training_env['current_host']
            }
        )
    
        if len(sm_hosts) > 1:
            # Wait until all hosts are able to find each other
            entry_point._wait_hostname_resolution()
    
            # Execute training function after initializing rabit.
            distributed.rabit_run(
                exec_fun=_xgb_train,
                args=xgb_train_args,
                include_in_training=(dtrain is not None),
                hosts=sm_hosts,
                current_host=sm_current_host,
                update_rabit_args=True
            )
        else:
            # If single node training, call training method directly.
            if dtrain:
                xgb_train_args['is_master'] = True
                _xgb_train(**xgb_train_args)
            else:
                raise ValueError("Training channel must have data to train model.")

# Return model object
def model_fn(model_dir):
    """Deserialize and return fitted model.

    Note that this should have the same name as the serialized model in the _xgb_train method
    """
    model_file = 'xgboost-model'
    booster = pkl.load(open(os.path.join(model_dir, model_file), 'rb'))
    return booster

Overwriting ./training/train.py


In [57]:
packages = ['mlflow', 'sagemaker-mlflow']
requirements = [f'{p}=={version(p)}' for p in packages]

if requirements:
    with open(f'./training/requirements.txt', 'w') as f:
        f.write('\n'.join(requirements))
    print("\nNew requirements.txt file created with the following content:")
    print('\n'.join(requirements))
else:
    print("\nNo requirements.txt file created as no packages were found")


New requirements.txt file created with the following content:
mlflow==2.22.2
sagemaker-mlflow==0.1.1


In [58]:
s3_input_train = sagemaker.inputs.TrainingInput(train_s3_url, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(validation_s3_url, content_type='csv')

training_inputs = {'train': s3_input_train, 'validation': s3_input_validation}

In [59]:
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

# Define where the training job stores the model artifact
output_s3_url = f"s3://{bucket_name}/{bucket_prefix}/output"

%store output_s3_url

Stored 'output_s3_url' (str)


In [60]:
hyperparams = {
    'num_round': 50,
    'max_depth': 3,
    'eta': 0.5,
    'alpha': 2.5,
    'objective': 'binary:logistic',
    'eval_metric': 'auc',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'min_child_weight': 3,
    'early_stopping_rounds': 10,
    'verbosity': 1
}

user_profile_name = 'byuMLGrad'

env_variables = {
    'MLFLOW_TRACKING_ARN': 'http://localhost:5000',
    'MLFLOW_EXPERIMENT_NAME': experiment_name,
    'USER': user_profile_name,
    'REGION': 'us-east-1'
}

In [62]:
%cp -rf ./training/* ./training-locals

from sagemaker.xgboost.estimator import XGBoost
from sagemaker.local import LocalSession

LOCAL_SESSION = LocalSession()
LOCAL_SESSION.config = {'local': {'local_code': True}}  # Ensure full code locality, see: https://sagemaker.readthedocs.io/en/stable/overview.html#local-mode

xgb_script_mode_local = XGBoost(
    entry_point='train.py',
    source_dir='./training-local',
    framework_version="1.7-1",  # Note: framework_version is mandatory
    hyperparameters=hyperparams,
    role=sagemaker.get_execution_role(),
    instance_count=train_instance_count,
    instance_type='local',
    output_path=output_s3_url,
    base_job_name="from-idea-to-prod-training",
    environment=env_variables,
    sagemaker_session=LOCAL_SESSION,
)

xgb_script_mode_local.fit(
    training_inputs,
    wait=True,
    logs=True,
)

# get the last run in MLflow
last_run_id = mlflow.search_runs(
    experiment_ids=[mlflow.get_experiment_by_name(experiment_name).experiment_id], 
    max_results=1, 
    order_by=["attributes.start_time DESC"]
)['run_id'][0]

# get the presigned url to open the MLflow UI
presigned_url = sm.create_presigned_mlflow_tracking_server_url(
    TrackingServerName=mlflow_name,
    ExpiresInSeconds=60,
    SessionExpirationDurationInSeconds=1800
)['AuthorizedUrl']

mlflow_run_link = f"{presigned_url.split('/auth')[0]}/#/experiments/1/runs/{last_run_id}"

display(Javascript('window.open("{}");'.format(presigned_url)))


cp: target './training-locals' is not a directory


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:23                                                                                   │
│                                                                                                  │
│   20 │   sagemaker_session=LOCAL_SESSION,                                                        │
│   21 )                                                                                           │
│   22                                                                                             │
│ ❱ 23 xgb_script_mode_local.fit(                                                                  │
│   24 │   training_inputs,                                                                        │
│   25 │   wait=True,                                                                              │
│   26 │   logs=True,                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:171 in wrapper  │
│                                                                                                  │
│   168 │   │   │   │   │   caught_ex = e                                                          │
│   169 │   │   │   │   finally:                                                                   │
│   170 │   │   │   │   │   if caught_ex:                                                          │
│ ❱ 171 │   │   │   │   │   │   raise caught_ex                                                    │
│   172 │   │   │   │   │   return response  # pylint: disable=W0150                               │
│   173 │   │   │   else:                                                                          │
│   174 │   │   │   │   logger.debug(                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py:142 in wrapper  │
│                                                                                                  │
│   139 │   │   │   │   start_timer = perf_counter()                                               │
│   140 │   │   │   │   try:                                                                       │
│   141 │   │   │   │   │   # Call the original function                                           │
│ ❱ 142 │   │   │   │   │   response = func(*args, **kwargs)                                       │
│   143 │   │   │   │   │   stop_timer = perf_counter()                                            │
│   144 │   │   │   │   │   elapsed = stop_timer - start_timer                                     │
│   145 │   │   │   │   │   extra += f"&x-latency={round(elapsed, 2)}"                             │
│                                                                                                  │
│ /opt/conda/lib/python3.12/site-packages/sagemaker/workflow/pipeline_context.py:346 in wrapper    │
│                                                                                                  │
│   343 │   │   │                                                                                  │
│   344 │   │   │   return _StepArguments(retrieve_caller_name(self_instance), run_func, *args,    │
│   345 │   │                                                                                      │
│ ❱ 346 │   │   return run_func(*args, **kwargs)                                                   │
│   347 │                                                                                          │
│   348 │   return wrapper                                                                         │
│   349                                                      

In [ ]:
display(Javascript('window.open("{}");'.format(mlflow_run_link)))

In [ ]:
from sagemaker.xgboost.estimator import XGBoost

xgb_script_mode_managed = XGBoost(
    entry_point='train.py',
    source_dir='./training',
    framework_version="1.7-1",  
    hyperparameters=hyperparams,
    role=sm_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    output_path=output_s3_url,
    base_job_name="from-idea-to-prod-training",
    environment=env_variables,
)

### Python SDK estimator classes
SageMaker Python SDK contains corresponding [`EstimatorBase`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase)-derived classes to access each of the built-in algorithms. You can extend [`Framework`](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.Framework) class to implement a training with a custom framework.

![](../img/python-sdk-estimators.png)

In [55]:
# Create an estimator
train_instance_count = 1
train_instance_type = "ml.m5.xlarge"

training_image = sagemaker.image_uris.retrieve("xgboost", region=region, version="1.7-1")

estimator = sagemaker.estimator.Estimator(
    image_uri=training_image,  # XGBoost algorithm container
    instance_type=train_instance_type,  # type of training instance
    instance_count=train_instance_count,  # number of instances to be used
    role=sm_role,  # IAM execution role to be used
    max_run=20 * 60,  # Maximum allowed active runtime
    # use_spot_instances=True,  # Use spot instances to reduce cost
    # max_wait=30 * 60,  # Maximum clock time (including spot delays)
    output_path=output_s3_url, # S3 location for saving the training result
    sagemaker_session=session, # Session object which manages interactions with SageMaker API and AWS services
    base_job_name="from-idea-to-prod-training", # Prefix for training job name
)

# define its hyperparameters
estimator.set_hyperparameters(
    num_round=50, # the number of rounds to run the training
    max_depth=3, # maximum depth of a tree
    eta=0.5, # step size shrinkage used in updates to prevent overfitting
    alpha=2.5, # L1 regularization term on weights
    objective="binary:logistic",
    eval_metric="auc", # evaluation metrics for validation data
    subsample=0.8, # subsample ratio of the training instance
    colsample_bytree=0.8, # subsample ratio of columns when constructing each tree
    min_child_weight=3, # minimum sum of instance weight (hessian) needed in a child
    early_stopping_rounds=10, # the model trains until the validation score stops improving
    verbosity=1, # verbosity of printing messages

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 train_instance_count = 1                                                                     │
│   3 train_instance_type = "ml.m5.xlarge"                                                         │
│   4                                                                                              │
│ ❱ 5 estimator = sagemaker.estimator.Estimator()                                                  │
│   6                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
TypeError: Estimator.__init__() missing 1 required positional argument: 'image_uri'

In [ ]:
# Set hyperparameters for the estimator algorithm
# estimator.set_hyperparameters()

In [ ]:
# Set the training inputs
# training_inputs = {}

In [ ]:
# Run the training job, optionally use an experiment_config parameter
# estimator.fit(training_inputs)

Wait until the training job is done.

In [ ]:
# Describe the training job
# training_job_name = estimator._current_job_name
# boto3.client("sagemaker", region_name=region).describe_training_job(TrainingJobName=training_job_name)

In [ ]:
# Get the model metrics from the describe job result

# print(f"Train-auc:{train_auc:.2f}, Validate-auc:{validate_auc:.2f}")

## Exercise 3: Validate model
To validate the model, you use the model artifact from the training job to run predictions on the test dataset. You can either create a [real-time inference endpoint](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html) or create a [batch transform](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).

### Option 1: Real-time inference
- Use [Estimator.deploy](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.deploy) function to provision a real-time inference endpoint
- Load test dataset
- Send the test dataset to the endpoint. Use [Predictor.predict](https://sagemaker.readthedocs.io/en/stable/api/inference/predictors.html#sagemaker.predictor.Predictor.predict) function
- Evaluate the predictions

In [ ]:
# Create a predictor
# Remember, the training job saved the test dataset in the specified S3 location

# predictor = estimator.deploy()

In [ ]:
# Load the test dataset
# test_x = pd.read_csv()
# test_y = pd.read_csv()

In [ ]:
# Predict


In [ ]:
# Evaluate predictions
# Compare the predicted label to ground truth label


### Option 2: Batch transform
For an asynchronous inference you can use a SageMaker [transform job](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform.html).
- Use [Estimator.tranformer](https://sagemaker.readthedocs.io/en/stable/api/training/estimators.html#sagemaker.estimator.EstimatorBase.transformer) function to create a transformer
- [Run](https://sagemaker.readthedocs.io/en/stable/api/inference/transformer.html#sagemaker.transformer.Transformer.transform) a tranform job
- Download the dataset from an S3 output location
- Evalute the predictions

In [ ]:
# Create a transformer
# transformer = estimator.transformer()

In [ ]:
# Run a transform, use an experiment_config parameter
# transformer.transform()

Wait until the transform job is done.

The transformer outputs the prediction probabilities and stores them as a `csv` file in the specified S3 location. The S3 path is stored in `transformer.output_path`. To compare the predictions with the ground truth labels, you must download the dataset from S3 and load it into a Pandas DataFrame.

In [ ]:
# Download the predictions and the ground truth labels from S3


In [ ]:
# Load the output dataset and the ground truth label
# predictions = pd.read_csv()
# test_y = pd.read_csv()

In [ ]:
# Show the confusion matrix
# pd.crosstab()


In [ ]:
# Calculate AUC
# test_auc = roc_auc_score(test_y, predictions)
#  print(f"Test-auc: {test_auc:.2f}")

### [Optional] build ROC and precision-recall curves
You can create various charts using [`sklearn.metrics`](https://scikit-learn.org/stable/modules/model_evaluation.html) package.

### [Optional] Save charts to the trial component
You can use the Tracker class to save various charts to a trial component of the trial of your experiment.

A Jupyter notebook trick: Press `Ctrl` + `/` to comment or uncomment all selected lines in the sell.

In [ ]:
# Find a trial component name of the current trial based on the display name
#batch_transform_trail_component = [
#    tc for tc in trial.list_trial_components() 
#    if tc.display_name == <DISPLAY NAME OF THE TRIAL COMPONENT>][0]

In [ ]:
# Add charts
# with Tracker.load(
#    trial_component_name=batch_transform_trail_component.trial_component_name,
#    sagemaker_boto_client=sm
# ) as tracker:
#    tracker.log_precision_recall()
#    tracker.log_confusion_matrix()
#    tracker.log_roc_curve()

### [Optional] Explore experiments, trials, and trial components in Studio
In **SageMaker resources** select **Experiment and trials**, choose **Open in trial component list** from the context menu:

<img src="../img/experiment-and-trials-context-menu.png" width="400"/>

## Exercise 4: [Optional] hyperparameter optimization (HPO)
- Use [HyperparameterTuner](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.HyperparameterTuner) to run a HPO job
- Specify hyperparameters ranges and tuning strategy
- [Run](https://sagemaker.readthedocs.io/en/stable/api/training/tuner.html#sagemaker.tuner.HyperparameterTuner.fit) the tuning job
- Compare performance of the tuned and non-tuned models

In [ ]:
# import required HPO objects
from sagemaker.tuner import (
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
    IntegerParameter,
)

In [ ]:
# set up hyperparameter ranges
# hp_ranges = {}


In [ ]:
# set up the objective metric
objective = "validation:auc"

In [ ]:
# instantiate a HPO object
# tuner = HyperparameterTuner()

In [ ]:
# evaluate performance

## Clean-up
Remove all real-time endpoints you created

In [ ]:
# predictor.delete_endpoint(delete_endpoint_config=True)


In [ ]:
# run if you created a tuned predictor after HPO
# hpo_predictor.delete_endpoint(delete_endpoint_config=True)


## Continue with the assignment 3
Navigate to the [assignment 3](03-assignment-sagemaker-pipeline.ipynb) notebook.